# Parameterized Queries

As discussed in class, many Visual Analytics systems function as wrappers around database queries, including those designed for graph data. In this part of the assignment, you will implement a parameterized query, allowing users to customize query inputs dynamically. This will help you understand how parameterization work, which is the start of implementing your own system in the next part of the assignment.

In [1]:
import os
import sys
import networkx as nx
import ipywidgets as widgets
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
from utils import setup_database

In [2]:
# Connect to existing database from part 1
connection = setup_database('../tmp', delete_existing=False)

Loading graph database
Removing lock file
Removed stale lock file: ../tmp/.lock


## Building Parameterized Graph Query
1. Set up the output area (already done for you)
2. Make a function that constructs a parameterized query
3. Use that function to query the database
4. Visualize the query results
5. Build interaction elements to configure parameterized query

In [3]:
output_area = widgets.Output(layout={'border': '1px solid black', 'min_height': '300px'})

In [4]:
# TODO: construct a Cypher query with a parameter to filter nodes or relationships in the graph.
def construct_query(parameter_value, operator):
    return 'MATCH (n) RETURN n LIMIT 10'

In [5]:
# Define a function to execute the query, process the results, and visualize the data
@output_area.capture(clear_output=True)
def visualize_results(parameter_value, operator):
    plt.close('all')
    
    # Use the parameter_value and operator to construct a Cypher query and execute it
    query = construct_query(parameter_value, operator)
    results = connection.execute(query)
    
    # Process the results into a Graph
    graph = results.get_as_networkx(directed=False)

    if graph.number_of_nodes() == 0:
        print(f"No results found for parameter value: {parameter_value}")
        return
    
    print(f"Graph has {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges")

    plt.figure(figsize=(10, 8))
    ax = plt.gca()
    pos = nx.spring_layout(graph, seed=42)


    nx.draw(graph, pos, 
            ax=ax,
            with_labels=False, 
            edge_color='gray',
            arrowsize=10,
            alpha=0.8)

    plt.title(f"Movie Network (Movies with year {operator} {parameter_value})")
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    plt.tight_layout()
    plt.show()

In [6]:
# Create an interactive widgets for dropdown and slider
operator_dropdown = widgets.Dropdown(
    options=['>=', '<=', '=', '>', '<'],
    value='>=',
    description='Operator:',
    disabled=False,
)

slider = widgets.IntSlider(
    value=2000,
    min=1950,
    max=2024,
    step=1,
    description='Parameter:',
    continuous_update=False 
)

In [7]:
# Display the initial visualization with the default slider value
widgets.interactive(visualize_results, parameter_value=slider, operator=operator_dropdown)

display(widgets.VBox([
    slider,
    operator_dropdown,
    output_area
    ]))